In [1]:
!pip install pyomo -q
!pip install mistralai -q
!wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00


In [2]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [3]:
API_KEY = ''

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 3
MODEL_TEMPERATURE = 0.7

SYSTEM_PROMPT_1 = """Please formulate a mathematical optimization model for this problem.
It is important that you do this by following these steps:

1. Define parameters and variables
2. Define the objective function
3. Define the constraints

Please note that each answer is always dedicated to one step. Therefore, do not provide extra information than what is asked. The answers will be manually combined by the user to create the full model."""
SYSTEM_PROMPT_2 = """Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."""
USER_PROMPT_1 = """We are now at the first step."""
USER_PROMPT_2 = """We are now at the second step."""
USER_PROMPT_3 = """We are now at the third step."""
PROBLEM_DESCRIPTION = """"A firm that packs refreshments and beers, situated in the province of Valencia (Spain) employs the same syrup to produce its 1.5 l COLI and PEPSA products on its S1 production line. Once processed, each hectolitre of syrup produces 40 units of the 1.5 l COLI product and 20 units of the 1.5 l PEPSA product. If X1 units of the 1.5 l COLI product and X2 units of the 1.5 l PEPSA product are produced, the firm estimates that the daily income obtained in dollars would be given by the following function:
49000 times X1 minus X1 squared plus 30 times X2 minus two times X2 squared.
It costs 150 dollars to buy and process each hectolitre of syrup. The S1 packaging line has a net capacity of producing 7100 1.5 l product units every hour. The firm works 5 days a week in 8h shifts. Given its weekly target coverage, the firm is committed to produce at least half the amount of PEPSA than COLI. Although priority orders tend to amend its production planning, the firm wishes to have a basic product planning that optimises its daily profits."""


client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-06-14 08:24:04.076538


## Step 1 - Generate Mathematical Formulation

In [4]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION),
    ChatMessage(role="user", content=USER_PROMPT_1)
]

response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_1_text = response_1.choices[0].message.content

In [5]:
Markdown(response_1_text)

Step 1: Define parameters and variables

Parameters:
- $C_{syrup}$: Cost to buy and process each hectolitre of syrup = $150 (dollars/hectolitre)
- $P_{COLA}$: Production of 1.5 l COLI product per hectolitre of syrup = 40 (units/hectolitre)
- $P_{PEPSA}$: Production of 1.5 l PEPSA product per hectolitre of syrup = 20 (units/hectolitre)
- $Cap_{S1}$: Net capacity of S1 production line = 7100 (units/hour)
- $Hours$: Working hours per day = 8 (hours/day)
- $Days$: Working days per week = 5 (days/week)

Variables:
- $X1$: Units of 1.5 l COLI product produced per day
- $X2$: Units of 1.5 l PEPSA product produced per day

These parameters and variables will be used in the subsequent steps to formulate the mathematical optimization model.

In [6]:
print(response_1_text)

Step 1: Define parameters and variables

Parameters:
- $C_{syrup}$: Cost to buy and process each hectolitre of syrup = $150 (dollars/hectolitre)
- $P_{COLA}$: Production of 1.5 l COLI product per hectolitre of syrup = 40 (units/hectolitre)
- $P_{PEPSA}$: Production of 1.5 l PEPSA product per hectolitre of syrup = 20 (units/hectolitre)
- $Cap_{S1}$: Net capacity of S1 production line = 7100 (units/hour)
- $Hours$: Working hours per day = 8 (hours/day)
- $Days$: Working days per week = 5 (days/week)

Variables:
- $X1$: Units of 1.5 l COLI product produced per day
- $X2$: Units of 1.5 l PEPSA product produced per day

These parameters and variables will be used in the subsequent steps to formulate the mathematical optimization model.


In [7]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION),
    ChatMessage(role="user", content=USER_PROMPT_1),
    ChatMessage(role="assistant", content=response_1_text),
    ChatMessage(role="user", content=USER_PROMPT_2),
]

response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [8]:
Markdown(response_2_text)

Step 2: Define the objective function

The objective function represents the daily income obtained by the firm in dollars. The given function for daily income is:

$Income = 49000 \cdot X1 - X1^2 + 30 \cdot X2 - 2 \cdot X2^2$

Where $X1$ is the number of units of the 1.5 l COLI product produced daily, and $X2$ is the number of units of the 1.5 l PEPSA product produced daily.

This function represents the revenue obtained from selling the COLI and PEPSA products minus the associated costs (which are quadratic functions of the production quantities). The goal is to maximize this function to find the optimal production plan that generates the highest daily income.

In [9]:
print(response_2_text)

Step 2: Define the objective function

The objective function represents the daily income obtained by the firm in dollars. The given function for daily income is:

$Income = 49000 \cdot X1 - X1^2 + 30 \cdot X2 - 2 \cdot X2^2$

Where $X1$ is the number of units of the 1.5 l COLI product produced daily, and $X2$ is the number of units of the 1.5 l PEPSA product produced daily.

This function represents the revenue obtained from selling the COLI and PEPSA products minus the associated costs (which are quadratic functions of the production quantities). The goal is to maximize this function to find the optimal production plan that generates the highest daily income.


In [10]:
messages_3 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION),
    ChatMessage(role="user", content=USER_PROMPT_1),
    ChatMessage(role="assistant", content=response_1_text),
    ChatMessage(role="user", content=USER_PROMPT_2),
    ChatMessage(role="assistant", content=response_2_text),
    ChatMessage(role="user", content=USER_PROMPT_3),
]

response_3 = client.chat(
    model=MODEL_ID,
    messages=messages_3,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_3_text = response_3.choices[0].message.content

In [11]:
Markdown(response_3_text)

Step 3: Define the constraints

The constraints represent the limitations and conditions that the firm must satisfy while producing the COLI and PEPSA products. The given constraints are:

1. Capacity constraint: The production of COLI and PEPSA products should not exceed the net capacity of the S1 production line.

$X1 + X2 \leq Cap_{S1} \cdot Hours$

2. Syrup capacity constraint: The amount of syrup used to produce the COLI and PEPSA products should not exceed the available syrup.

$\frac{X1}{P_{COLA}} + \frac{X2}{P_{PEPSA}} \leq \frac{Income}{C_{syrup}}$

3. Proportional production constraint: The firm is committed to producing at least half the amount of PEPSA than COLI.

$X2 \geq \frac{1}{2} \cdot X1$

These constraints ensure that the production quantities are feasible and meet the requirements of the firm.

Now, we have defined the mathematical optimization model for the problem. The model consists of the objective function and the constraints, which can be solved using mathematical optimization techniques to find the optimal production plan that maximizes the daily income.

In [12]:
print(response_3_text)

Step 3: Define the constraints

The constraints represent the limitations and conditions that the firm must satisfy while producing the COLI and PEPSA products. The given constraints are:

1. Capacity constraint: The production of COLI and PEPSA products should not exceed the net capacity of the S1 production line.

$X1 + X2 \leq Cap_{S1} \cdot Hours$

2. Syrup capacity constraint: The amount of syrup used to produce the COLI and PEPSA products should not exceed the available syrup.

$\frac{X1}{P_{COLA}} + \frac{X2}{P_{PEPSA}} \leq \frac{Income}{C_{syrup}}$

3. Proportional production constraint: The firm is committed to producing at least half the amount of PEPSA than COLI.

$X2 \geq \frac{1}{2} \cdot X1$

These constraints ensure that the production quantities are feasible and meet the requirements of the firm.

Now, we have defined the mathematical optimization model for the problem. The model consists of the objective function and the constraints, which can be solved using mathemat

In [13]:
response_text = '\n'.join([response_1_text, response_2_text, response_3_text])

Markdown(response_text)

Step 1: Define parameters and variables

Parameters:
- $C_{syrup}$: Cost to buy and process each hectolitre of syrup = $150 (dollars/hectolitre)
- $P_{COLA}$: Production of 1.5 l COLI product per hectolitre of syrup = 40 (units/hectolitre)
- $P_{PEPSA}$: Production of 1.5 l PEPSA product per hectolitre of syrup = 20 (units/hectolitre)
- $Cap_{S1}$: Net capacity of S1 production line = 7100 (units/hour)
- $Hours$: Working hours per day = 8 (hours/day)
- $Days$: Working days per week = 5 (days/week)

Variables:
- $X1$: Units of 1.5 l COLI product produced per day
- $X2$: Units of 1.5 l PEPSA product produced per day

These parameters and variables will be used in the subsequent steps to formulate the mathematical optimization model.
Step 2: Define the objective function

The objective function represents the daily income obtained by the firm in dollars. The given function for daily income is:

$Income = 49000 \cdot X1 - X1^2 + 30 \cdot X2 - 2 \cdot X2^2$

Where $X1$ is the number of units of the 1.5 l COLI product produced daily, and $X2$ is the number of units of the 1.5 l PEPSA product produced daily.

This function represents the revenue obtained from selling the COLI and PEPSA products minus the associated costs (which are quadratic functions of the production quantities). The goal is to maximize this function to find the optimal production plan that generates the highest daily income.
Step 3: Define the constraints

The constraints represent the limitations and conditions that the firm must satisfy while producing the COLI and PEPSA products. The given constraints are:

1. Capacity constraint: The production of COLI and PEPSA products should not exceed the net capacity of the S1 production line.

$X1 + X2 \leq Cap_{S1} \cdot Hours$

2. Syrup capacity constraint: The amount of syrup used to produce the COLI and PEPSA products should not exceed the available syrup.

$\frac{X1}{P_{COLA}} + \frac{X2}{P_{PEPSA}} \leq \frac{Income}{C_{syrup}}$

3. Proportional production constraint: The firm is committed to producing at least half the amount of PEPSA than COLI.

$X2 \geq \frac{1}{2} \cdot X1$

These constraints ensure that the production quantities are feasible and meet the requirements of the firm.

Now, we have defined the mathematical optimization model for the problem. The model consists of the objective function and the constraints, which can be solved using mathematical optimization techniques to find the optimal production plan that maximizes the daily income.

In [14]:
print(response_text)

Step 1: Define parameters and variables

Parameters:
- $C_{syrup}$: Cost to buy and process each hectolitre of syrup = $150 (dollars/hectolitre)
- $P_{COLA}$: Production of 1.5 l COLI product per hectolitre of syrup = 40 (units/hectolitre)
- $P_{PEPSA}$: Production of 1.5 l PEPSA product per hectolitre of syrup = 20 (units/hectolitre)
- $Cap_{S1}$: Net capacity of S1 production line = 7100 (units/hour)
- $Hours$: Working hours per day = 8 (hours/day)
- $Days$: Working days per week = 5 (days/week)

Variables:
- $X1$: Units of 1.5 l COLI product produced per day
- $X2$: Units of 1.5 l PEPSA product produced per day

These parameters and variables will be used in the subsequent steps to formulate the mathematical optimization model.
Step 2: Define the objective function

The objective function represents the daily income obtained by the firm in dollars. The given function for daily income is:

$Income = 49000 \cdot X1 - X1^2 + 30 \cdot X2 - 2 \cdot X2^2$

Where $X1$ is the number of unit

## Step 2 - Generate the Pyomo Code

In [15]:
messages_4 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_text)
]

In [16]:
response_4 = client.chat(
    model=MODEL_ID,
    messages=messages_4,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_4_text = response_4.choices[0].message.content

In [17]:
Markdown(response_4_text)

To solve the optimization problem using Pyomo, we first need to install the Pyomo package if it's not already installed. You can do this by running `!pip install pyomo` in a Jupyter notebook or `pip install pyomo` in your command line.

Here's the Python code using Pyomo to solve the optimization problem:

```python
import pyomo.environ as pyo

# Parameters (sample data)
C_syrup = 150  # Cost to buy and process each hectolitre of syrup (dollars/hectolitre)
P_COLA = 40  # Production of 1.5 l COLI product per hectolitre of syrup (units/hectolitre)
P_PEPSA = 20  # Production of 1.5 l PEPSA product per hectolitre of syrup (units/hectolitre)
Cap_S1 = 7100  # Net capacity of S1 production line (units/hour)
Hours = 8  # Working hours per day (hours/day)
Days = 5  # Working days per week (days/week)

# Initialize the Pyomo model
model = pyo.ConcreteModel()

# Define variables
model.X1 = pyo.Var(domain=pyo.NonNegativeIntegers)  # Units of 1.5 l COLI product produced per day
model.X2 = pyo.Var(domain=pyo.NonNegativeIntegers)  # Units of 1.5 l PEPSA product produced per day

# Objective function
def obj_rule(model):
    return 49000 * model.X1 - model.X1**2 + 30 * model.X2 - 2 * model.X2**2

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

# Constraints
def capacity_constraint_rule(model):
    return model.X1 + model.X2 <= Cap_S1 * Hours

model.capacity_constraint = pyo.Constraint(rule=capacity_constraint_rule)

def syrup_capacity_constraint_rule(model):
    return model.X1 / P_COLA + model.X2 / P_PEPSA <= (49000 * model.X1 - model.X1**2 + 30 * model.X2 - 2 * model.X2**2) / C_syrup

model.syrup_capacity_constraint = pyo.Constraint(rule=syrup_capacity_constraint_rule)

def proportional_production_constraint_rule(model):
    return model.X2 >= 0.5 * model.X1

model.proportional_production_constraint = pyo.Constraint(rule=proportional_production_constraint_rule)

# Solve the model
solver = pyo.SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print("Optimal daily production plan:")
print(f"COLI: {pyo.value(model.X1)} units")
print(f"PEPSA: {pyo.value(model.X2)} units")
print(f"Maximum daily income: ${pyo.value(model.obj)}")
```

This code defines the parameters and variables, sets up the objective function, and adds the constraints to the Pyomo model. Then, it uses the GLPK solver to find the optimal solution and prints the results.

In [18]:
print(response_4_text)

To solve the optimization problem using Pyomo, we first need to install the Pyomo package if it's not already installed. You can do this by running `!pip install pyomo` in a Jupyter notebook or `pip install pyomo` in your command line.

Here's the Python code using Pyomo to solve the optimization problem:

```python
import pyomo.environ as pyo

# Parameters (sample data)
C_syrup = 150  # Cost to buy and process each hectolitre of syrup (dollars/hectolitre)
P_COLA = 40  # Production of 1.5 l COLI product per hectolitre of syrup (units/hectolitre)
P_PEPSA = 20  # Production of 1.5 l PEPSA product per hectolitre of syrup (units/hectolitre)
Cap_S1 = 7100  # Net capacity of S1 production line (units/hour)
Hours = 8  # Working hours per day (hours/day)
Days = 5  # Working days per week (days/week)

# Initialize the Pyomo model
model = pyo.ConcreteModel()

# Define variables
model.X1 = pyo.Var(domain=pyo.NonNegativeIntegers)  # Units of 1.5 l COLI product produced per day
model.X2 = pyo.Var(d

### Code Executability

In [19]:
import pyomo.environ as pyo

# Parameters (sample data)
C_syrup = 150  # Cost to buy and process each hectolitre of syrup (dollars/hectolitre)
P_COLA = 40  # Production of 1.5 l COLI product per hectolitre of syrup (units/hectolitre)
P_PEPSA = 20  # Production of 1.5 l PEPSA product per hectolitre of syrup (units/hectolitre)
Cap_S1 = 7100  # Net capacity of S1 production line (units/hour)
Hours = 8  # Working hours per day (hours/day)
Days = 5  # Working days per week (days/week)

# Initialize the Pyomo model
model = pyo.ConcreteModel()

# Define variables
model.X1 = pyo.Var(domain=pyo.NonNegativeIntegers)  # Units of 1.5 l COLI product produced per day
model.X2 = pyo.Var(domain=pyo.NonNegativeIntegers)  # Units of 1.5 l PEPSA product produced per day

# Objective function
def obj_rule(model):
    return 49000 * model.X1 - model.X1**2 + 30 * model.X2 - 2 * model.X2**2

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

# Constraints
def capacity_constraint_rule(model):
    return model.X1 + model.X2 <= Cap_S1 * Hours

model.capacity_constraint = pyo.Constraint(rule=capacity_constraint_rule)

def syrup_capacity_constraint_rule(model):
    return model.X1 / P_COLA + model.X2 / P_PEPSA <= (49000 * model.X1 - model.X1**2 + 30 * model.X2 - 2 * model.X2**2) / C_syrup

model.syrup_capacity_constraint = pyo.Constraint(rule=syrup_capacity_constraint_rule)

def proportional_production_constraint_rule(model):
    return model.X2 >= 0.5 * model.X1

model.proportional_production_constraint = pyo.Constraint(rule=proportional_production_constraint_rule)

# Solve the model
solver = pyo.SolverFactory('ipopt')
results = solver.solve(model)

# Print the results
print("Optimal daily production plan:")
print(f"COLI: {pyo.value(model.X1)} units")
print(f"PEPSA: {pyo.value(model.X2)} units")
print(f"Maximum daily income: ${pyo.value(model.obj)}")

Optimal daily production plan:
COLI: 16338.333333340006 units
PEPSA: 8169.16666666004 units
Maximum daily income: $400411704.1669919


### Solution Correctness